In [1]:
pip install llama-index

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/3c/ff/ac74735884031a3b9ddf7b1abecee0885ec61660588b1e7c6862bccf5116/chromadb-0.4.14-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/11/7a/673ccb9bb2faf9cf655d9040e970c02a96645966e06837fde7d10edf242a/chroma_hnswlib-0.7.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/4d/d2/3ad038a2365fefbac19d9a046cab7ce45f4c7bfa81d877cbece9707de9ce/fastapi-0.103.2-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/79/96/b0882a1c3f7ef3dd86879e041212ae5b62b4bd352320889231cc735a8e8f/uvicorn-0.23.2-py3-none-any.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c66a766

In [2]:
pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/f6/c8/ad19a273d6e825fcd040f51b559d30f55d652d82f5badf1f546a9a06aeb9/pyarrow-13.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/9c/0a/d0fd8d78c8a2c3c3b34e7a9dccf85f01bf38f32e0228d107fa3903e0981f/xxhash-3.4.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/e7/41/96ac938770ba6e7d5ae1d8c9cafebac54b413549042c6260f0

In [4]:
# import
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from IPython.display import Markdown, display
import chromadb


In [6]:
from datasets import load_dataset
dataset = load_dataset("santoshtyss/indian_courts_cases")

/Users/harsha/miniconda3/envs/precidentAIHack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 28816
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3000
    })
})

In [7]:
dataset["train"][:10]

{'text': ['CIVIL APPELLATE JURISDICTION Civil Appeal No. 3838 of 1988. From the Judgment and Order dated 22.6.1987 of the Tamil Nadu High Court in W.A. No. 555 of 1984. T.S. Tulsi, Additional Solicitor General, B.K. Prasad, K. Srivastava, P. Parmeshwaran for the Appellants. T.M. Sampath and K.V. Sreekumar for the Respondents. The Judgment of the Court was delivered by VERMA, J. This matter brings to the fore once again the ineptitude with which litigation is companyducted quite often on behalf of the Government of India and State Governments even when important issues having lasting and wide repercussions are involved. The point in this case relates to the validity of a policy of the railway administration and is likely to affect the staff pattern in several units. Inspite of this fact, to support validity of the impugned policy the required materials were number produced in the High Court and to overcome the adverse decision several opportunities given by us to produce the entire rele

In [8]:
from llama_index import Document
documents = []
for entry in dataset["train"]:
    text_data = entry['text']
    documents.append(Document(text=text_data))

In [9]:
smaller_documents = documents[:10]

In [10]:
# save to disk
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    smaller_documents, storage_context=storage_context
)

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("what does the document say about Harnam Singh?")
print(response)

The document states that Harnam Singh made a gift of agriculture land measuring 76 acres 3 bighas in favor of the appellant.


In [13]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
)